# Setup

In [1]:
 from IPython.display import clear_output
 !pip install jdatetime
 clear_output()

### Import Libraries

In [2]:
# standard libraries
import pandas as pd
import numpy as np


#visualizations
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


#math and statistic
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
import scipy.stats as stats


#machine learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

#persian datetime
import jdatetime


# Load Data

In [3]:
gold_data = pd.read_csv('gold_data.csv')
usd_data = pd.read_csv('usd_data.csv')
eur_data = pd.read_csv('eur_data.csv')

In [4]:
gold_data['date'] = pd.to_datetime(gold_data['timestamp'])
usd_data['date'] = pd.to_datetime(usd_data['timestamp'])
eur_data['date'] = pd.to_datetime(eur_data['timestamp'])

gold_data = gold_data.drop('timestamp', axis=1)
usd_data = usd_data.drop('timestamp', axis=1)
eur_data = eur_data.drop('timestamp', axis=1)

# Data Preprocess

In [5]:
gold_data = gold_data.rename(columns={'price': 'price'})
usd_data = usd_data.rename(columns={'price': 'price_usd'})
eur_data = eur_data.rename(columns={'price': 'price_eur'})

### Shamsi DateTime Calculation

In [6]:
def gregorian_to_shamsi(date):
    if pd.isna(date):
        return None

    g_date = pd.Timestamp(date)
    j_date = jdatetime.date.fromgregorian(
        year=g_date.year,
        month=g_date.month,
        day=g_date.day
    )
    # fromat as YYYY/MM/DD
    return j_date.strftime('%Y/%m/%d')

In [7]:
gold_data['shamsi_date'] = gold_data['date'].apply(gregorian_to_shamsi)
usd_data['shamsi_date'] = usd_data['date'].apply(gregorian_to_shamsi)
eur_data['shamsi_date'] = eur_data['date'].apply(gregorian_to_shamsi)

### Merge Dataset

In [8]:
market_data = gold_data.merge(usd_data, on=['date', 'shamsi_date'], suffixes=('', '_usd'))
market_data = market_data.merge(eur_data, on=['date', 'shamsi_date'], suffixes=('', '_eur'))


#sort by date
market_data = market_data.sort_values('date')

Extract Shamsi Date Year,Month,Day

In [9]:
def extract_shamsi_components(date_str):
    if pd.isna(date_str):
        return None, None, None

    parts = date_str.split('/')
    if len(parts) != 3:
        return None, None, None

    return int(parts[0]), int(parts[1]), int(parts[2])


market_data['shamsi_year'], market_data['shamsi_month'], market_data['shamsi_day'] = zip(
    *market_data['shamsi_date'].apply(lambda x: extract_shamsi_components(x))
)

In [10]:
def get_shamsi_weekday(date):
    if pd.isna(date):
        return None

    g_date = pd.Timestamp(date)
    j_date = jdatetime.date.fromgregorian(
        year=g_date.year,
        month=g_date.month,
        day=g_date.day
    )
    return (j_date.weekday() + 2) % 7

market_data['shamsi_weekday'] = market_data['date'].apply(get_shamsi_weekday)

#ma weekday numbers
shamsi_weekday_names = {
    0: 'شنبه',
    1: 'یکشنبه',
    2: 'دوشنبه',
    3: 'سه‌شنبه',
    4: 'چهارشنبه',
    5: 'پنجشنبه',
    6: 'جمعه'
}

market_data['shamsi_weekday_name'] = market_data['shamsi_weekday'].map(shamsi_weekday_names)

#ma month numbers
shamsi_month_names = {
    1: 'فروردین',
    2: 'اردیبهشت',
    3: 'خرداد',
    4: 'تیر',
    5: 'مرداد',
    6: 'شهریور',
    7: 'مهر',
    8: 'آبان',
    9: 'آذر',
    10: 'دی',
    11: 'بهمن',
    12: 'اسفند'
}

market_data['shamsi_month_name'] = market_data['shamsi_month'].map(shamsi_month_names)


def get_shamsi_season(month):
    if pd.isna(month):
        return None
    if month in [1, 2, 3]:
        return 'بهار'
    elif month in [4, 5, 6]:
        return 'تابستان'
    elif month in [7, 8, 9]:
        return 'پاییز'
    else:
        return 'زمستان'

market_data['shamsi_season'] = market_data['shamsi_month'].apply(get_shamsi_season)


market_data.set_index('date', inplace=True)
market_data.head()

,price,shamsi_date,price_usd,price_eur,shamsi_year,shamsi_month,shamsi_day,shamsi_weekday,shamsi_weekday_name,shamsi_month_name,shamsi_season
date,,,,,,,,,,,
2013-07-22,1000000,1392/04/31,31200,41600,1392,4,31,4,چهارشنبه,تیر,تابستان
2013-07-23,1030000,1392/05/01,31650,41850,1392,5,1,5,پنجشنبه,مرداد,تابستان
2013-07-24,1020000,1392/05/02,31600,41950,1392,5,2,6,جمعه,مرداد,تابستان
2013-07-25,1025000,1392/05/03,31670,42382,1392,5,3,0,شنبه,مرداد,تابستان
2013-07-27,1040000,1392/05/05,31800,42500,1392,5,5,2,دوشنبه,مرداد,تابستان


###‌ Filter Date ( Start From  Farvardin 97)

In [11]:
market_data = market_data[market_data.index >= '2018-03-21']

In [12]:
market_data.head()

,price,shamsi_date,price_usd,price_eur,shamsi_year,shamsi_month,shamsi_day,shamsi_weekday,shamsi_weekday_name,shamsi_month_name,shamsi_season
date,,,,,,,,,,,
2018-03-25,1620670,1397/01/05,51030,61710,1397,1,5,3,سه‌شنبه,فروردین,بهار
2018-03-26,1595740,1397/01/06,50490,62200,1397,1,6,4,چهارشنبه,فروردین,بهار
2018-03-27,1574270,1397/01/07,50460,61950,1397,1,7,5,پنجشنبه,فروردین,بهار
2018-03-28,1588580,1397/01/08,50450,61700,1397,1,8,6,جمعه,فروردین,بهار
2018-03-29,1591810,1397/01/09,51020,61900,1397,1,9,0,شنبه,فروردین,بهار


# Exploratory Data Analysis and Visualization

### Time Series Analysis


In [13]:
#main figure
fig = make_subplots(rows=3,
                    cols=1,
                    shared_xaxes=True,
                    subplot_titles=('<b>قیمت طلا (ریال)</b>', '<b>نرخ دلار به ریال</b>', '<b>نرخ یورو به ریال</b>'),
                    vertical_spacing=0.1)

#gold subplot
fig.add_trace(go.Scatter(
    x=market_data.index,
    y=market_data['price'],
    name='قیمت طلا',
    line=dict(color='gold', width=1.5)
), row=1, col=1)

#usd/irr subplot
fig.add_trace(go.Scatter(
    x=market_data.index,
    y=market_data['price_usd'],
    name='نرخ دلار',
    line=dict(color='green', width=1.5)
), row=2, col=1)

#eru/irr
fig.add_trace(go.Scatter(
    x=market_data.index,
    y=market_data['price_eur'],
    name='نرخ یورو',
    line=dict(color='blue', width=1.5)
), row=3, col=1)

#add overall trend lines (linear regression)
#------------------------------------------

#gold trend line
x_numeric_gold = np.arange(len(market_data))
slope_gold, intercept_gold, _, _, _ = stats.linregress(x_numeric_gold, market_data['price'])
trend_gold = intercept_gold + slope_gold * x_numeric_gold

fig.add_trace(go.Scatter(
    x=market_data.index,
    y=trend_gold,
    name='روند کلی طلا',
    line=dict(color='red', width=1, dash='dash'),
    opacity=0.7
), row=1, col=1)

#usd trend line
x_numeric_usd = np.arange(len(market_data))
slope_usd, intercept_usd, _, _, _ = stats.linregress(x_numeric_usd, market_data['price_usd'])
trend_usd = intercept_usd + slope_usd * x_numeric_usd

fig.add_trace(go.Scatter(
    x=market_data.index,
    y=trend_usd,
    name='روند کلی دلار',
    line=dict(color='red', width=1, dash='dash'),
    opacity=0.7
), row=2, col=1)

#eur trend line
x_numeric_eur = np.arange(len(market_data))
slope_eur, intercept_eur, _, _, _ = stats.linregress(x_numeric_eur, market_data['price_eur'])
trend_eur = intercept_eur + slope_eur * x_numeric_eur

fig.add_trace(go.Scatter(
    x=market_data.index,
    y=trend_eur,
    name='روند کلی یورو',
    line=dict(color='red', width=1, dash='dash'),
    opacity=0.7
), row=3, col=1)

# add moving averages (ma) for better classic insights
#--------------------------------------------
fig.add_trace(go.Scatter(
    x=market_data.index,
    y=market_data['price'].rolling(window=50).mean(),
    name='میانگین متحرک ۵۰ روزه طلا',
    line=dict(color='orange', width=1),
    opacity=0.7
), row=1, col=1)

# 200 day moving average for gold
fig.add_trace(go.Scatter(
    x=market_data.index,
    y=market_data['price'].rolling(window=200).mean(),
    name='میانگین متحرک ۲۰۰ روزه طلا',
    line=dict(color='purple', width=1),
    opacity=0.7
), row=1, col=1)

#yearly growth rates
yearly_data = market_data.resample('Y').last()
yearly_gold_growth = yearly_data['price'].pct_change() * 100
yearly_usd_growth = yearly_data['price_usd'].pct_change() * 100
yearly_eur_growth = yearly_data['price_eur'].pct_change() * 100

#total growth over the entire period
total_gold_growth = (market_data['price'].iloc[-1] / market_data['price'].iloc[0] - 1) * 100
total_usd_growth = (market_data['price_usd'].iloc[-1] / market_data['price_usd'].iloc[0] - 1) * 100
total_eur_growth = (market_data['price_eur'].iloc[-1] / market_data['price_eur'].iloc[0] - 1) * 100

#price annotations to the right side
fig.add_annotation(
    xref="paper",
    yref="y1",
    x=1.01,
    y=market_data['price'].iloc[-1],
    text=f"<b>{format(int(market_data['price'].iloc[-1]), ',')}</b> ریال",
    showarrow=False,
    font=dict(family="B Nazanin, Arial", size=10, color="gold"),
    align="right",
    row=1, col=1
)

fig.add_annotation(
    xref="paper",
    yref="y2",
    x=1.01,
    y=market_data['price_usd'].iloc[-1],
    text=f"<b>{format(int(market_data['price_usd'].iloc[-1]), ',')}</b> ریال",
    showarrow=False,
    font=dict(family="B Nazanin, Arial", size=10, color="green"),
    align="right",
    row=2, col=1
)

fig.add_annotation(
    xref="paper",
    yref="y3",
    x=1.01,
    y=market_data['price_eur'].iloc[-1],
    text=f"<b>{format(int(market_data['price_eur'].iloc[-1]), ',')}</b> ریال",
    showarrow=False,
    font=dict(family="B Nazanin, Arial", size=10, color="blue"),
    align="right",
    row=3, col=1
)

#key economic events line
key_events = {
    '2018-07-29': ('تشدید تحریم‌ها', 1),
    '2019-11-06': ('افزایش قیمت بنزین', 1),
    '2020-03-20': ('شیوع کرونا در ایران', 1),
    '2021-08-06': ('تغییر دولت', 1),
    '2022-02-24': ('جنگ روسیه و اوکراین', 1),
    '2022-05-12': ('پایان دلار ۴۲۰۰ تومانی', 1)

}

colors = ['purple', 'orange', 'brown', 'magenta', 'teal']


event_dates = [pd.Timestamp(date) for date in key_events.keys()]

#find closest dates in the dataset for each event
for i, (event_date, (event_text, row_num)) in enumerate(zip(event_dates, key_events.values())):
    #closest date in dataset
    closest_idx = np.argmin([abs((date - event_date).total_seconds()) for date in market_data.index])
    closest_date = market_data.index[closest_idx]

    #vertical line for all three subplots using shapes
    for row in range(1, 4):
        if row == 1:
            y_min, y_max = min(market_data['price']), max(market_data['price'])
        elif row == 2:
            y_min, y_max = min(market_data['price_usd']), max(market_data['price_usd'])
        else:
            y_min, y_max = min(market_data['price_eur']), max(market_data['price_eur'])

        fig.add_shape(
            type="line",
            x0=closest_date,
            y0=y_min,
            x1=closest_date,
            y1=y_max,
            line=dict(color=colors[i % len(colors)], width=1.5, dash="dash"),
            row=row, col=1
        )

    #annotation only to specified row
    y_position = market_data.iloc[closest_idx]['price' if row_num == 1 else
                                               'price_usd' if row_num == 2 else 'price_eur']

    fig.add_annotation(
        x=closest_date,
        y=y_position,
        text=event_text,
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-40 if row_num == 1 else 40,
        row=row_num, col=1,
        font=dict(color=colors[i % len(colors)], size=10, family="B Nazanin, Arial"),
        bgcolor="white",
        bordercolor=colors[i % len(colors)],
        borderwidth=1,
        opacity=0.9
    )

#x axis to show dates across entire range
date_ticks = pd.date_range(start=market_data.index.min(), end=market_data.index.max(), periods=10)

#date ticks to shamsi format for display
date_labels = []
for date in date_ticks:
    jdate = jdatetime.date.fromgregorian(date=date.date())
    date_labels.append(f"{jdate.year}/{jdate.month:02d}")

# update x axes to ensure proper display across date range
for i in range(1, 4):
    fig.update_xaxes(
        range=[market_data.index.min(), market_data.index.max()],
        tickangle=0,
        tickmode='array',
        tickvals=date_ticks,
        ticktext=date_labels,
        tickfont=dict(family="B Nazanin, Arial", size=10),
        row=i, col=1
    )


fig.update_xaxes(
    title_text="<b>تاریخ شمسی</b>",
    title_font=dict(family="B Nazanin, Arial", size=12),
    row=3, col=1
)


fig.update_yaxes(
    tickformat=",.0f",
    title_text="<b>ریال</b>",
    title_font=dict(family="B Nazanin, Arial", size=12),
    tickfont=dict(family="B Nazanin, Arial", size=10),
    row=1, col=1,
    side="right",
    range=[0, market_data['price'].max() * 1.05]
)

fig.update_yaxes(
    tickformat=",.0f",
    title_text="<b>ریال</b>",
    title_font=dict(family="B Nazanin, Arial", size=12),
    tickfont=dict(family="B Nazanin, Arial", size=10),
    row=2, col=1,
    side="right",
    range=[0, market_data['price_usd'].max() * 1.05]
)

fig.update_yaxes(
    tickformat=",.0f",
    title_text="<b>ریال</b>",
    title_font=dict(family="B Nazanin, Arial", size=12),
    tickfont=dict(family="B Nazanin, Arial", size=10),
    row=3, col=1,
    side="right",
    range=[0, market_data['price_eur'].max() * 1.05]
)

#summary box with key statistics
#-------------------------------
start_date_shamsi = jdatetime.date.fromgregorian(date=market_data.index.min().date()).strftime('%Y/%m/%d')
end_date_shamsi = jdatetime.date.fromgregorian(date=market_data.index.max().date()).strftime('%Y/%m/%d')

summary_text = (
    f"<b>تحلیل بازار از {start_date_shamsi} تا {end_date_shamsi}</b><br>" +
    f"<b>رشد کل طلا:</b> {total_gold_growth:.1f}% | <b>متوسط رشد سالانه:</b> {yearly_gold_growth.mean():.1f}%<br>" +
    f"<b>رشد کل دلار:</b> {total_usd_growth:.1f}% | <b>متوسط رشد سالانه:</b> {yearly_usd_growth.mean():.1f}%<br>" +
    f"<b>رشد کل یورو:</b> {total_eur_growth:.1f}% | <b>متوسط رشد سالانه:</b> {yearly_eur_growth.mean():.1f}%<br>" +
    f"<b>نسبت تغییرات طلا به دلار:</b> {(total_gold_growth/total_usd_growth):.2f}x"
)

#summary box
fig.add_annotation(
    xref="paper",
    yref="paper",
    x=0.5,
    y=1.18,
    text=summary_text,
    showarrow=False,
    font=dict(family="B Nazanin, Arial", size=12),
    bgcolor="rgba(255, 255, 255, 0.9)",
    bordercolor="black",
    borderwidth=1,
    align="center"
)



#highlight periods of significant gold vs currency divergence
gold_usd_ratio = market_data['price'] / market_data['price_usd']
ratio_mean = gold_usd_ratio.mean()
ratio_std = gold_usd_ratio.std()

#where gold significantly outperformed or underperformed usd
divergence_periods = market_data[(gold_usd_ratio > ratio_mean + 2*ratio_std) |
                                 (gold_usd_ratio < ratio_mean - 2*ratio_std)]

if not divergence_periods.empty:
    max_divergence_idx = divergence_periods.index[0]

    fig.add_annotation(
        x=max_divergence_idx,
        y=market_data.loc[max_divergence_idx, 'price'],
        text="واگرایی قیمت طلا و ارز",
        showarrow=True,
        arrowhead=2,
        ax=40,
        ay=-40,
        row=1, col=1,
        font=dict(color="blue", size=10, family="B Nazanin, Arial"),
        bgcolor="white",
        opacity=0.9
    )

fig.update_layout(
    title={
        'text': '<b>تحلیل تاریخی قیمت طلا، دلار و یورو در بازار ایران</b>',
        'font': dict(family="B Nazanin, Arial", size=18),
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    template='plotly_white',
    width=1200,
    height=900,
    font=dict(family="B Nazanin, Arial", size=12),
    hoverlabel=dict(font_family="B Nazanin, Arial"),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.15,
        xanchor="center",
        x=0.5,
        font=dict(family="B Nazanin, Arial", size=12)
    ),
    margin=dict(l=50, r=100, t=150, b=150)
)


fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.update_layout(
    xaxis=dict(
        rangeslider=dict(visible=False),
        type="date"
    ),

    modebar=dict(
        remove=[
           #'zoom', 'pan', 'select', 'zoomIn', 'zoomOut','resetScale','toImage',
           # 'autoScale',  'lasso2d',
           # 'sendDataToCloud', 'toggleSpikelines', 'hoverClosestCartesian',
           # 'hoverCompareCartesian'
        ]
    )
)


config = {'displayModeBar': True, 'staticPlot': False}


fig.show(config=config)

<ipython-input-13-a6577b5bfaae>:94: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



# Assets Return And Correlation Analysis


In [14]:
# calculate returns for better correlation analysis
market_data['gold_return'] = market_data['price'].pct_change()
market_data['usd_return'] = market_data['price_usd'].pct_change()
market_data['eur_return'] = market_data['price_eur'].pct_change()


returns_data = market_data.dropna()

#correlation heatmap
corr_matrix = returns_data[['gold_return', 'usd_return', 'eur_return']].rename(columns={"gold_return":"بازده طلا","usd_return":"بازده دلار","eur_return":"بازده یورو"}).corr()

fig = px.imshow(
    corr_matrix ,
    text_auto=True,
    color_continuous_scale='RdBu_r',
    title='<b>همبستگی بازده بین طلا ،دلار و یورو </b>',
)

fig.update_layout(
    width=700,
    height=600,
    template='plotly_white',
    title_x=0.5
)

fig.show()

### Gold Price in USD and EUR Terms


In [15]:
#gold price in usd and eur terms
market_data['gold_in_usd'] = market_data['price'] / market_data['price_usd']
market_data['gold_in_eur'] = market_data['price'] / market_data['price_eur']

#moving averages to see trends more clearly
market_data['gold_usd_ma50'] = market_data['gold_in_usd'].rolling(window=50).mean()
market_data['gold_usd_ma200'] = market_data['gold_in_usd'].rolling(window=200).mean()
market_data['gold_eur_ma50'] = market_data['gold_in_eur'].rolling(window=50).mean()
market_data['gold_eur_ma200'] = market_data['gold_in_eur'].rolling(window=200).mean()

#percentage change from the beginning of period
gold_usd_change = ((market_data['gold_in_usd'].iloc[-1] / market_data['gold_in_usd'].iloc[0]) - 1) * 100
gold_eur_change = ((market_data['gold_in_eur'].iloc[-1] / market_data['gold_in_eur'].iloc[0]) - 1) * 100

#main figure
fig = make_subplots(rows=2, cols=1,
                    shared_xaxes=True,
                    subplot_titles=('قیمت طلا بر حسب دلار', 'قیمت طلا بر حسب یورو'),
                    vertical_spacing=0.15)


fig.add_trace(go.Scatter(
    x=market_data.index,
    y=market_data['gold_in_usd'],
    name='قیمت طلا (دلار)',
    line=dict(color='darkgoldenrod', width=1.5)
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=market_data.index,
    y=market_data['gold_in_eur'],
    name='قیمت طلا (یورو)',
    line=dict(color='darkorange', width=1.5)
), row=2, col=1)

#moving averages
fig.add_trace(go.Scatter(
    x=market_data.index,
    y=market_data['gold_usd_ma50'],
    name='میانگین متحرک ۵۰ روزه (دلار)',
    line=dict(color='blue', width=1, dash='dash')
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=market_data.index,
    y=market_data['gold_usd_ma200'],
    name='میانگین متحرک ۲۰۰ روزه (دلار)',
    line=dict(color='purple', width=1, dash='dash')
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=market_data.index,
    y=market_data['gold_eur_ma50'],
    name='میانگین متحرک ۵۰ روزه (یورو)',
    line=dict(color='blue', width=1, dash='dash')
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=market_data.index,
    y=market_data['gold_eur_ma200'],
    name='میانگین متحرک ۲۰۰ روزه (یورو)',
    line=dict(color='purple', width=1, dash='dash')
), row=2, col=1)

#key events
key_events = {
    '2018-05-08': ('خروج آمریکا از برجام', 1),
    '2020-01-03': ('شهادت سردار سلیمانی', 1),
    '2020-03-11': ('اعلام پاندمی کرونا', 2),
    '2022-02-24': ('جنگ روسیه و اوکراین', 2)
}

colors = ['red', 'green', 'orange', 'magenta']


for i, (date_str, (event_text, row_num)) in enumerate(key_events.items()):
    try:
        event_date = pd.Timestamp(date_str)
        if event_date >= market_data.index.min() and event_date <= market_data.index.max():
            closest_idx = min(range(len(market_data.index)), key=lambda i: abs((market_data.index[i] - event_date).total_seconds()))
            closest_date = market_data.index[closest_idx]


            fig.add_shape(
                type="line",
                x0=closest_date,
                y0=0,
                x1=closest_date,
                y1=1,
                xref="x",
                yref="paper",
                line=dict(color=colors[i % len(colors)], width=1.5, dash="dash"),
                row=row_num, col=1
            )

            y_position = market_data.iloc[closest_idx]['gold_in_usd' if row_num == 1 else 'gold_in_eur']

            fig.add_annotation(
                x=closest_date,
                y=y_position,
                text=event_text,
                showarrow=True,
                arrowhead=2,
                ax=0,
                ay=-40,
                row=row_num, col=1,
                font=dict(color=colors[i % len(colors)], size=10, family="B Nazanin, Arial"),
                bgcolor="white",
                opacity=0.8
            )
    except Exception as e:
        print(f"Error adding event {date_str}: {e}")

#calculate gold to currency ratio statistics
start_period = market_data.index[0].strftime('%Y-%m-%d')
end_period = market_data.index[-1].strftime('%Y-%m-%d')
mean_gold_usd = market_data['gold_in_usd'].mean()
mean_gold_eur = market_data['gold_in_eur'].mean()
current_gold_usd = market_data['gold_in_usd'].iloc[-1]
current_gold_eur = market_data['gold_in_eur'].iloc[-1]

#calculate if current value is over/under long term average
usd_status = "بالاتر" if current_gold_usd > mean_gold_usd else "پایین‌تر"
eur_status = "بالاتر" if current_gold_eur > mean_gold_eur else "پایین‌تر"
usd_percent = abs((current_gold_usd / mean_gold_usd - 1) * 100)
eur_percent = abs((current_gold_eur / mean_gold_eur - 1) * 100)


start_shamsi = jdatetime.date.fromgregorian(date=pd.Timestamp(start_period).date()).strftime('%Y/%m/%d')
end_shamsi = jdatetime.date.fromgregorian(date=pd.Timestamp(end_period).date()).strftime('%Y/%m/%d')

#summary statistics as annotations
fig.add_annotation(
    xref="paper",
    yref="paper",
    x=0.5,
    y=1.2,
    text=(f"<b>تحلیل ارزش طلا بر اساس ارزهای خارجی</b><br>" +
          f"دوره زمانی: {start_shamsi} تا {end_shamsi}<br>" +
          f"تغییر ارزش طلا بر حسب دلار: {gold_usd_change:.2f}% | بر حسب یورو: {gold_eur_change:.2f}%<br>" +
          f"میانگین تاریخی نسبت طلا به دلار: {mean_gold_usd:.2f} | نسبت فعلی: {current_gold_usd:.2f} ({usd_percent:.1f}% {usd_status} از میانگین)<br>" +
          f"میانگین تاریخی نسبت طلا به یورو: {mean_gold_eur:.2f} | نسبت فعلی: {current_gold_eur:.2f} ({eur_percent:.1f}% {eur_status} از میانگین)"),
    showarrow=False,
    font=dict(family="B Nazanin, Arial", size=12),
    bgcolor="rgba(255, 255, 255, 0.9)",
    bordercolor="black",
    borderwidth=1,
    align="center"
)

date_ticks = pd.date_range(start=market_data.index.min(), end=market_data.index.max(), freq='6MS')
shamsi_tick_labels = [gregorian_to_shamsi(date) for date in date_ticks]

shamsi_tick_labels = [date.split('/')[0] + '/' + date.split('/')[1] for date in shamsi_tick_labels]

#update x axis to show shamsi dates
for i in range(1, 3):
    fig.update_xaxes(
        title_text="تاریخ شمسی" if i == 2 else None,
        tickvals=date_ticks,
        ticktext=shamsi_tick_labels,
        tickangle=45,
        title_font=dict(family="B Nazanin, Arial", size=12),
        tickfont=dict(family="B Nazanin, Arial", size=10),
        row=i, col=1
    )


fig.update_yaxes(title_text="قیمت طلا (دلار)", row=1, col=1, title_font=dict(family="B Nazanin, Arial", size=12))
fig.update_yaxes(title_text="قیمت طلا (یورو)", row=2, col=1, title_font=dict(family="B Nazanin, Arial", size=12))

fig.update_layout(
    title={
        'text': '<b>تحلیل قیمت طلا بر اساس نرخ ارزهای خارجی</b>',
        'font': dict(family="B Nazanin, Arial", size=18),
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5,
        font=dict(family="B Nazanin, Arial", size=10)
    ),
    height=700,
    template='plotly_white',
    font=dict(family="B Nazanin, Arial"),
    margin=dict(l=50, r=50, t=150, b=150)
)

config = {'displayModeBar': False}
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show(config=config)

### Volatility Analysis

In [16]:

window = 30
market_data['gold_volatility'] = market_data['gold_return'].rolling(window).std() * np.sqrt(252)
market_data['usd_volatility'] = market_data['usd_return'].rolling(window).std() * np.sqrt(252)
market_data['eur_volatility'] = market_data['eur_return'].rolling(window).std() * np.sqrt(252)


avg_gold_vol = market_data['gold_volatility'].mean()
avg_usd_vol = market_data['usd_volatility'].mean()
avg_eur_vol = market_data['eur_volatility'].mean()


max_gold_vol_date = market_data.loc[market_data['gold_volatility'].idxmax(), 'shamsi_date']
max_usd_vol_date = market_data.loc[market_data['usd_volatility'].idxmax(), 'shamsi_date']
max_eur_vol_date = market_data.loc[market_data['eur_volatility'].idxmax(), 'shamsi_date']

max_gold_vol = market_data['gold_volatility'].max()
max_usd_vol = market_data['usd_volatility'].max()
max_eur_vol = market_data['eur_volatility'].max()


fig = go.Figure()


fig.add_trace(go.Scatter(
    x=market_data.index,
    y=market_data['gold_volatility'],
    name='نوسان قیمت طلا',
    line=dict(color='gold', width=2),
    hovertemplate='<b>تاریخ:</b> %{text}<br><b>نوسان طلا:</b> %{y:.2f}%<extra></extra>',
    text=market_data['shamsi_date']
))


fig.add_trace(go.Scatter(
    x=market_data.index,
    y=market_data['usd_volatility'],
    name='نوسان نرخ دلار',
    line=dict(color='green', width=2),
    hovertemplate='<b>تاریخ:</b> %{text}<br><b>نوسان دلار:</b> %{y:.2f}%<extra></extra>',
    text=market_data['shamsi_date']
))


fig.add_trace(go.Scatter(
    x=market_data.index,
    y=market_data['eur_volatility'],
    name='نوسان نرخ یورو',
    line=dict(color='blue', width=2),
    hovertemplate='<b>تاریخ:</b> %{text}<br><b>نوسان یورو:</b> %{y:.2f}%<extra></extra>',
    text=market_data['shamsi_date']
))

fig.add_trace(go.Scatter(
    x=[market_data.index[0], market_data.index[-1]],
    y=[avg_gold_vol, avg_gold_vol],
    name='میانگین نوسان طلا',
    line=dict(color='gold', width=1, dash='dash'),
    opacity=0.7
))

fig.add_trace(go.Scatter(
    x=[market_data.index[0], market_data.index[-1]],
    y=[avg_usd_vol, avg_usd_vol],
    name='میانگین نوسان دلار',
    line=dict(color='green', width=1, dash='dash'),
    opacity=0.7
))

fig.add_trace(go.Scatter(
    x=[market_data.index[0], market_data.index[-1]],
    y=[avg_eur_vol, avg_eur_vol],
    name='میانگین نوسان یورو',
    line=dict(color='blue', width=1, dash='dash'),
    opacity=0.7
))


for asset, color, max_date, max_val in [
    ('طلا', 'gold', market_data['gold_volatility'].idxmax(), max_gold_vol),
    ('دلار', 'green', market_data['usd_volatility'].idxmax(), max_usd_vol),
    ('یورو', 'blue', market_data['eur_volatility'].idxmax(), max_eur_vol)
]:

    if not pd.isna(max_date):
        fig.add_trace(go.Scatter(
            x=[max_date],
            y=[max_val],
            mode='markers',
            marker=dict(color=color, size=10, line=dict(width=2, color='white')),
            name=f'بیشترین نوسان {asset}',
            showlegend=False
        ))

        shamsi_date = market_data.loc[max_date, 'shamsi_date']
        fig.add_annotation(
            x=max_date,
            y=max_val,
            text=f"بیشترین نوسان {asset}<br>{shamsi_date}",
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-40,
            font=dict(color=color, size=10, family="B Nazanin, Arial"),
            bgcolor="white",
            opacity=0.8
        )


date_ticks = pd.date_range(start=market_data.index.min(), end=market_data.index.max(), freq='6MS')
shamsi_tick_labels = []

for date in date_ticks:
    idx = market_data.index.get_indexer([date], method='nearest')[0]
    if idx >= 0 and idx < len(market_data):
        shamsi_date = market_data['shamsi_date'].iloc[idx]
        shamsi_tick_labels.append('/'.join(shamsi_date.split('/')[:2]))
    else:
        shamsi_tick_labels.append('')


fig.update_layout(
    title={
        'text': '<b>نوسان‌ ۳۰ روزه بازارهای طلا و ارز</b>',
        'font': dict(family="B Nazanin, Arial", size=18),
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis={
        'title': 'تاریخ',
        'titlefont': dict(family="B Nazanin, Arial", size=14),
        'tickfont': dict(family="B Nazanin, Arial", size=12),
        'tickvals': date_ticks,
        'ticktext': shamsi_tick_labels,
        'tickangle': 45
    },
    yaxis={
        'title': 'شاخص نوسان سالانه شده (٪)',
        'titlefont': dict(family="B Nazanin, Arial", size=14),
        'tickfont': dict(family="B Nazanin, Arial", size=12),
        'ticksuffix': '%'
    },
    legend={
        'title': 'دارایی',
        'title_font': dict(family="B Nazanin, Arial", size=12),
        'font': dict(family="B Nazanin, Arial", size=12),
        'orientation': 'h',
        'yanchor': 'bottom',
        'y': -0.2,
        'xanchor': 'center',
        'x': 0.5
    },
    template='plotly_white',
    height=700,
    margin=dict(l=50, r=50, t=100, b=150)
)


start_shamsi = market_data['shamsi_date'].iloc[0]
end_shamsi = market_data['shamsi_date'].iloc[-1]

fig.add_annotation(
    xref="paper",
    yref="paper",
    x=0.5,
    y=1.10,
    text=(f"<b>مقایسه نوسان‌  بازار طلا و ارز در بازار ایران</b><br>" +
          f"دوره زمانی: {start_shamsi} تا {end_shamsi}<br>" +
          f"میانگین نوسان طلا: {avg_gold_vol:.2f}% | دلار: {avg_usd_vol:.2f}% | یورو: {avg_eur_vol:.2f}%<br>" +
          f"بیشترین نوسان طلا: {max_gold_vol:.2f}% ({max_gold_vol_date}) | دلار: {max_usd_vol:.2f}% ({max_usd_vol_date})"),
    showarrow=False,
    font=dict(family="B Nazanin, Arial", size=12),
    bgcolor="rgba(255, 255, 255, 0.9)",
    bordercolor="black",
    borderwidth=1,
    align="center"
)


config = {'displayModeBar': False}
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)


fig.show(config=config)

### Technical Indicators For On Last Year

In [27]:
merged_data = market_data.copy()

# ema calculations
merged_data['gold_ema_10'] = merged_data['price'].ewm(span=10, adjust=False).mean()
merged_data['gold_ema_30'] = merged_data['price'].ewm(span=30, adjust=False).mean()
merged_data['gold_ema_50'] = merged_data['price'].ewm(span=50, adjust=False).mean()

# rsi
def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = delta.where(delta > 0, 0).rolling(window=period).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

merged_data['gold_rsi'] = calculate_rsi(merged_data['price'])

# macd calculations
merged_data['gold_ema_12'] = merged_data['price'].ewm(span=12, adjust=False).mean()
merged_data['gold_ema_26'] = merged_data['price'].ewm(span=26, adjust=False).mean()
merged_data['gold_macd'] = merged_data['gold_ema_12'] - merged_data['gold_ema_26']
merged_data['gold_signal'] = merged_data['gold_macd'].ewm(span=9, adjust=False).mean()
merged_data['gold_macd_hist'] = merged_data['gold_macd'] - merged_data['gold_signal']

# bb
merged_data['gold_ma_20'] = merged_data['price'].rolling(window=20).mean()
merged_data['gold_std_20'] = merged_data['price'].rolling(window=20).std()
merged_data['gold_upper_band'] = merged_data['gold_ma_20'] + (merged_data['gold_std_20'] * 2)
merged_data['gold_lower_band'] = merged_data['gold_ma_20'] - (merged_data['gold_std_20'] * 2)


# just show last yaer for better insight
recent_data = merged_data.iloc[-365:].copy()


start_date_shamsi = recent_data['shamsi_date'].iloc[0]
end_date_shamsi = recent_data['shamsi_date'].iloc[-1]

#main figure
fig = make_subplots(rows=4, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.05,
                    subplot_titles=('قیمت طلا با باندهای بولینگر',
                                   'شاخص قدرت نسبی (RSI)',
                                   'واگرایی و همگرایی میانگین متحرک (MACD)',
                                   'حجم تغییرات روزانه'),
                    row_heights=[0.5, 0.15, 0.15, 0.2])

#gold and bb
fig.add_trace(go.Scatter(x=recent_data.index, y=recent_data['price'],
                        name='قیمت طلا', line=dict(color='gold', width=2),
                        hovertemplate='<b>تاریخ:</b> %{text}<br><b>قیمت:</b> %{y:,.0f} ریال<extra></extra>',
                        text=recent_data['shamsi_date']),
            row=1, col=1)

fig.add_trace(go.Scatter(x=recent_data.index, y=recent_data['gold_upper_band'],
                        name='باند بالایی', line=dict(color='rgba(173, 204, 255, 0.7)'),
                        hovertemplate='<b>تاریخ:</b> %{text}<br><b>باند بالایی:</b> %{y:,.0f} ریال<extra></extra>',
                        text=recent_data['shamsi_date']),
            row=1, col=1)

fig.add_trace(go.Scatter(x=recent_data.index, y=recent_data['gold_lower_band'],
                        name='باند پایینی', line=dict(color='rgba(173, 204, 255, 0.7)'),
                        fill='tonexty',
                        hovertemplate='<b>تاریخ:</b> %{text}<br><b>باند پایینی:</b> %{y:,.0f} ریال<extra></extra>',
                        text=recent_data['shamsi_date']),
            row=1, col=1)

fig.add_trace(go.Scatter(x=recent_data.index, y=recent_data['gold_ma_20'],
                        name='میانگین متحرک 20 روزه', line=dict(color='blue', width=1),
                        hovertemplate='<b>تاریخ:</b> %{text}<br><b>میانگین 20 روزه:</b> %{y:,.0f} ریال<extra></extra>',
                        text=recent_data['shamsi_date']),
            row=1, col=1)

# rsi
fig.add_trace(go.Scatter(x=recent_data.index, y=recent_data['gold_rsi'],
                        name='RSI', line=dict(color='purple'),
                        hovertemplate='<b>تاریخ:</b> %{text}<br><b>RSI:</b> %{y:.2f}<extra></extra>',
                        text=recent_data['shamsi_date']),
            row=2, col=1)

#rsi lines
fig.add_shape(type="line", x0=recent_data.index[0], y0=70, x1=recent_data.index[-1], y1=70,
            line=dict(color="red", width=1, dash="dash"), row=2, col=1)
fig.add_annotation(x=recent_data.index[0], y=70, text="اشباع خرید",
                  showarrow=False, xshift=40, yshift=10,
                  font=dict(size=10, color="red", family="B Nazanin, Arial"),
                  row=2, col=1)

fig.add_shape(type="line", x0=recent_data.index[0], y0=30, x1=recent_data.index[-1], y1=30,
            line=dict(color="green", width=1, dash="dash"), row=2, col=1)
fig.add_annotation(x=recent_data.index[0], y=30, text="اشباع فروش",
                  showarrow=False, xshift=40, yshift=-10,
                  font=dict(size=10, color="green", family="B Nazanin, Arial"),
                  row=2, col=1)

#macd
fig.add_trace(go.Scatter(x=recent_data.index, y=recent_data['gold_macd'],
                        name='MACD', line=dict(color='blue'),
                        hovertemplate='<b>تاریخ:</b> %{text}<br><b>MACD:</b> %{y:.2f}<extra></extra>',
                        text=recent_data['shamsi_date']),
            row=3, col=1)

fig.add_trace(go.Scatter(x=recent_data.index, y=recent_data['gold_signal'],
                        name='سیگنال', line=dict(color='red'),
                        hovertemplate='<b>تاریخ:</b> %{text}<br><b>سیگنال:</b> %{y:.2f}<extra></extra>',
                        text=recent_data['shamsi_date']),
            row=3, col=1)


colors = ['red' if val < 0 else 'green' for val in recent_data['gold_macd_hist']]

fig.add_trace(go.Bar(x=recent_data.index, y=recent_data['gold_macd_hist'],
                    name='هیستوگرام', marker_color=colors,
                    hovertemplate='<b>تاریخ:</b> %{text}<br><b>هیستوگرام:</b> %{y:.2f}<extra></extra>',
                    text=recent_data['shamsi_date']),
            row=3, col=1)

# daily chnage
fig.add_trace(go.Bar(x=recent_data.index, y=abs(recent_data['gold_return']),
                    name='بزرگی تغییرات', marker_color='gold',
                    hovertemplate='<b>تاریخ:</b> %{text}<br><b>تغییر روزانه:</b> %{y:.2f}%<extra></extra>',
                    text=recent_data['shamsi_date']),
            row=4, col=1)

date_ticks = pd.date_range(start=recent_data.index.min(), end=recent_data.index.max(), freq='1MS')
shamsi_tick_labels = []

for date in date_ticks:
    idx = recent_data.index.get_indexer([date], method='nearest')[0]
    if idx >= 0 and idx < len(recent_data):
        shamsi_date = recent_data['shamsi_date'].iloc[idx]

        parts = shamsi_date.split('/')
        if len(parts) >= 2:
            shamsi_tick_labels.append(f"{parts[0]}/{parts[1]}")
        else:
            shamsi_tick_labels.append(shamsi_date)

#buy and sell signals
rsi_buy_signals = recent_data[(recent_data['gold_rsi'] < 30) & (recent_data['gold_rsi'].shift(1) >= 30)]
rsi_sell_signals = recent_data[(recent_data['gold_rsi'] > 70) & (recent_data['gold_rsi'].shift(1) <= 70)]

# macd signals
macd_buy_signals = recent_data[(recent_data['gold_macd'] > recent_data['gold_signal']) &
                               (recent_data['gold_macd'].shift(1) <= recent_data['gold_signal'].shift(1))]
macd_sell_signals = recent_data[(recent_data['gold_macd'] < recent_data['gold_signal']) &
                                (recent_data['gold_macd'].shift(1) >= recent_data['gold_signal'].shift(1))]

# rsi signals
for idx, row in rsi_buy_signals.iterrows():
    fig.add_trace(go.Scatter(
        x=[idx],
        y=[row['gold_rsi']],
        mode='markers',
        marker=dict(symbol='triangle-up', size=10, color='green'),
        name='سیگنال خرید RSI',
        showlegend=False,
        hoverinfo='text',
        hovertext=f"سیگنال خرید RSI<br>{row['shamsi_date']}<br>RSI: {row['gold_rsi']:.2f}"
    ), row=2, col=1)

for idx, row in rsi_sell_signals.iterrows():
    fig.add_trace(go.Scatter(
        x=[idx],
        y=[row['gold_rsi']],
        mode='markers',
        marker=dict(symbol='triangle-down', size=10, color='red'),
        name='سیگنال فروش RSI',
        showlegend=False,
        hoverinfo='text',
        hovertext=f"سیگنال فروش RSI<br>{row['shamsi_date']}<br>RSI: {row['gold_rsi']:.2f}"
    ), row=2, col=1)


for idx, row in macd_buy_signals.iterrows():
    fig.add_trace(go.Scatter(
        x=[idx],
        y=[row['gold_macd']],
        mode='markers',
        marker=dict(symbol='triangle-up', size=10, color='green'),
        name='سیگنال خرید MACD',
        showlegend=False,
        hoverinfo='text',
        hovertext=f"سیگنال خرید MACD<br>{row['shamsi_date']}<br>MACD: {row['gold_macd']:.2f}"
    ), row=3, col=1)

for idx, row in macd_sell_signals.iterrows():
    fig.add_trace(go.Scatter(
        x=[idx],
        y=[row['gold_macd']],
        mode='markers',
        marker=dict(symbol='triangle-down', size=10, color='red'),
        name='سیگنال فروش MACD',
        showlegend=False,
        hoverinfo='text',
        hovertext=f"سیگنال فروش MACD<br>{row['shamsi_date']}<br>MACD: {row['gold_macd']:.2f}"
    ), row=3, col=1)


fig.update_layout(
    title={
        'text': f'<b>داشبورد تحلیل تکنیکال طلا</b>  <span style="font-size:14px">دوره: {start_date_shamsi} تا {end_date_shamsi}</span>',
        'font': dict(family="B Nazanin, Arial", size=20),
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    height=1200,
    width=1200,
    template='plotly_white',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.15,
        xanchor="center",
        x=0.5,
        font=dict(family="B Nazanin, Arial", size=12)
    ),
    font=dict(family="B Nazanin, Arial", size=12),
    hoverlabel=dict(font_family="B Nazanin, Arial")
)



fig.update_yaxes(title_text='قیمت (ریال)', row=1, col=1, title_font=dict(family="B Nazanin, Arial", size=14), tickformat=',d')
fig.update_yaxes(title_text='RSI', row=2, col=1, title_font=dict(family="B Nazanin, Arial", size=14))
fig.update_yaxes(title_text='MACD', row=3, col=1, title_font=dict(family="B Nazanin, Arial", size=14))
fig.update_yaxes(title_text='درصد تغییرات', row=4, col=1, title_font=dict(family="B Nazanin, Arial", size=14))

# extra info
current_price = recent_data['price'].iloc[-1]
current_ma20 = recent_data['gold_ma_20'].iloc[-1]
current_rsi = recent_data['gold_rsi'].iloc[-1]
current_macd = recent_data['gold_macd'].iloc[-1]
current_signal = recent_data['gold_signal'].iloc[-1]

#current market
if current_price > current_ma20:
    price_status = "بالاتر از میانگین 20 روزه (روند صعودی)"
    price_color = "green"
else:
    price_status = "پایین‌تر از میانگین 20 روزه (روند نزولی)"
    price_color = "red"

if current_rsi > 70:
    rsi_status = "اشباع خرید (احتمال اصلاح قیمت)"
    rsi_color = "red"
elif current_rsi < 30:
    rsi_status = "اشباع فروش (احتمال افزایش قیمت)"
    rsi_color = "green"
else:
    rsi_status = "در محدوده نرمال"
    rsi_color = "black"

if current_macd > current_signal:
    macd_status = "MACD بالاتر از سیگنال (روند صعودی)"
    macd_color = "green"
else:
    macd_status = "MACD پایین‌تر از سیگنال (روند نزولی)"
    macd_color = "red"


fig.add_annotation(
    xref="paper", yref="paper",
    x=0.01, y=0.99,
    text=(f"<b>تحلیل وضعیت فعلی:</b><br>" +
          f"<span style='color:{price_color};'>• {price_status}</span><br>" +
          f"<span style='color:{rsi_color};'>• RSI = {current_rsi:.1f}: {rsi_status}</span><br>" +
          f"<span style='color:{macd_color};'>• {macd_status}</span>"),
    showarrow=False,
    align="left",
    bgcolor="rgba(255, 255, 255, 0.8)",
    bordercolor="black",
    borderwidth=1,
    font=dict(family="B Nazanin, Arial", size=12)
)


fig.add_annotation(
    xref="paper", yref="paper",
    x=0.99, y=0.99,
    text=("<b>راهنمای سیگنال‌ها:</b><br>" +
          "▲ سیگنال خرید<br>" +
          "▼ سیگنال فروش"),
    showarrow=False,
    align="right",
    bgcolor="rgba(255, 255, 255, 0.8)",
    bordercolor="black",
    borderwidth=1,
    font=dict(family="B Nazanin, Arial", size=12)
)

fig.show()

### Average Weekdays Returns Fro Gold/Usd/Eur

In [21]:
market_data['day_of_week'] = market_data.index.dayofweek
market_data['gold_return'] = market_data['price'].pct_change() * 100
market_data['usd_return'] = market_data['price_usd'].pct_change() * 100
market_data['eur_return'] = market_data['price_eur'].pct_change() * 100


market_data['shamsi_day_of_week'] = market_data.index.map(get_shamsi_weekday)

#average returns by shamsi day of week
avg_returns = market_data.groupby('shamsi_day_of_week')[['gold_return', 'usd_return', 'eur_return']].mean()

#std to show volatility
std_returns = market_data.groupby('shamsi_day_of_week')[['gold_return', 'usd_return', 'eur_return']].std()


persian_day_mapping = {
    0: 'شنبه',
    1: 'یکشنبه',
    2: 'دوشنبه',
    3: 'سه‌شنبه',
    4: 'چهارشنبه',
    5: 'پنج‌شنبه',
    6: 'جمعه'
}


days_order = list(range(7))
categories = [persian_day_mapping[i] for i in days_order]


r_gold = [avg_returns.loc[day, 'gold_return'] if day in avg_returns.index else 0 for day in days_order]
r_usd = [avg_returns.loc[day, 'usd_return'] if day in avg_returns.index else 0 for day in days_order]
r_eur = [avg_returns.loc[day, 'eur_return'] if day in avg_returns.index else 0 for day in days_order]

#polar plot
fig = go.Figure()


fig.add_trace(go.Scatterpolar(
    r=r_gold,
    theta=categories,
    fill='toself',
    name='بازدهی طلا',
    line=dict(color='gold', width=2),
    opacity=0.8
))


fig.add_trace(go.Scatterpolar(
    r=r_usd,
    theta=categories,
    fill='toself',
    name='بازدهی دلار',
    line=dict(color='green', width=2),
    opacity=0.8
))


fig.add_trace(go.Scatterpolar(
    r=r_eur,
    theta=categories,
    fill='toself',
    name='بازدهی یورو',
    line=dict(color='blue', width=2),
    opacity=0.7
))

#best and worst days for each asset
best_day_gold = persian_day_mapping[avg_returns['gold_return'].idxmax()]
worst_day_gold = persian_day_mapping[avg_returns['gold_return'].idxmin()]
best_day_usd = persian_day_mapping[avg_returns['usd_return'].idxmax()]
worst_day_usd = persian_day_mapping[avg_returns['usd_return'].idxmin()]

min_val = min(min(r_gold), min(r_usd), min(r_eur))
max_val = max(max(r_gold), max(r_usd), max(r_eur))
range_min = min_val - 0.1
range_max = max_val + 0.1


fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[range_min, range_max],
            tickfont=dict(family="B Nazanin, Arial", size=10),
            ticksuffix="%",
            title=dict(
                text="بازدهی روزانه (درصد)",
                font=dict(family="B Nazanin, Arial", size=12)
            )
        ),
        angularaxis=dict(
            tickfont=dict(family="B Nazanin, Arial", size=14, color="black"),
            rotation=90,
            direction="clockwise"
        )
    ),
    title=dict(
        text='<b>میانگین بازدهی روزانه بر اساس روزهای هفته (شمسی)</b>',
        font=dict(family="B Nazanin, Arial", size=18),
        y=0.95
    ),
    showlegend=True,
    legend=dict(
        font=dict(family="B Nazanin, Arial", size=12),
        orientation="h",
        yanchor="bottom",
        y=-0.1,
        xanchor="center",
        x=0.5
    ),
    template='plotly_white',
    width=850,
    height=750,
    title_x=0.5,
    margin=dict(l=50, r=50, t=100, b=100)
)

#insights as annotations
fig.add_annotation(
    text=f"<b>بهترین روز برای طلا:</b> {best_day_gold} ({avg_returns['gold_return'].max():.2f}%)<br>" +
         f"<b>بدترین روز برای طلا:</b> {worst_day_gold} ({avg_returns['gold_return'].min():.2f}%)<br>" +
         f"<b>بهترین روز برای دلار:</b> {best_day_usd} ({avg_returns['usd_return'].max():.2f}%)<br>" +
         f"<b>بدترین روز برای دلار:</b> {worst_day_usd} ({avg_returns['usd_return'].min():.2f}%)",
    xref="paper", yref="paper",
    x=0.5, y=-0.18,
    showarrow=False,
    font=dict(family="B Nazanin, Arial", size=12),
    align="center",
    bgcolor="rgba(255, 255, 255, 0.8)",
    bordercolor="black",
  borderwidth=1
)

fig.show()


### Table Result For Average Return

In [28]:
table = pd.DataFrame(index=days_order)
for day in days_order:
    table.loc[day, 'روز هفته'] = persian_day_mapping[day]
    if day in avg_returns.index:
        table.loc[day, 'بازدهی طلا (%)'] = avg_returns.loc[day, 'gold_return']
        table.loc[day, 'بازدهی دلار (%)'] = avg_returns.loc[day, 'usd_return']
        table.loc[day, 'بازدهی یورو (%)'] = avg_returns.loc[day, 'eur_return']
        table.loc[day, 'نوسان طلا (%)'] = std_returns.loc[day, 'gold_return']
        table.loc[day, 'نوسان دلار (%)'] = std_returns.loc[day, 'usd_return']
        table.loc[day, 'نوسان یورو (%)'] = std_returns.loc[day, 'eur_return']
    else:
        table.loc[day, 'بازدهی طلا (%)'] = 0
        table.loc[day, 'بازدهی دلار (%)'] = 0
        table.loc[day, 'بازدهی یورو (%)'] = 0
        table.loc[day, 'نوسان طلا (%)'] = 0
        table.loc[day, 'نوسان دلار (%)'] = 0
        table.loc[day, 'نوسان یورو (%)'] = 0

for col in table.columns:
    if col != 'روز هفته':
        table[col] = table[col].map('{:.2f}'.format)
print("\nجدول خلاصه بازدهی روزانه بر اساس روزهای هفته:")
table


جدول خلاصه بازدهی روزانه بر اساس روزهای هفته:


,روز هفته,بازدهی طلا (%),بازدهی دلار (%),بازدهی یورو (%),نوسان طلا (%),نوسان دلار (%),نوسان یورو (%)
0,شنبه,0.08,-0.61,-0.01,1.01,2.19,1.47
1,یکشنبه,0.00,0.00,0.00,0.00,0.00,0.00
2,دوشنبه,0.36,1.21,0.14,2.83,3.68,2.57
3,سه‌شنبه,0.19,0.21,0.28,2.16,2.10,2.27
4,چهارشنبه,0.15,0.20,0.21,2.06,1.93,2.13
5,پنج‌شنبه,0.23,-0.04,0.10,1.78,2.45,1.87
6,جمعه,0.21,0.07,0.22,1.60,1.95,1.58
